In [137]:
import os
from collections import namedtuple
from time import time

import numpy as np
import pandas as pd
from ortools.sat.python import cp_model
from matplotlib import pyplot as plt

from src import dataset, config

%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
np.random.seed(config.RANDOM_SEED)

In [138]:
product_dataset = dataset.ProductDataset(
    "data/products.pkl",
    config.num_products,
    config.pallet_lenght, 
    config.pallet_width, 
    config.max_product_height, 
    config.pallet_load
)
product_dataset.products.head()

,lenght,width,height,weight
0,136,79,459,72
1,244,365,491,11
2,96,410,600,4
3,299,119,477,15
4,128,20,502,55


In [130]:
ordered_products = 10
order = product_dataset.get_order(ordered_products)
order.head()

,id,lenght,width,height,weight
0,389,225,232,522,49
1,565,309,92,210,25
2,105,54,131,691,66
3,771,65,457,417,42
4,821,146,352,753,27


Pallet EUR:

|       |  $L\times W\times H (cm)$   | Load (kg)  |
|:-----:|:--------------------------:|:--------:|
| EUR 1 |  $80\times 120\times 14.5$ |  $2490$  |
| EUR 2 | $120\times 100\times 14.4$ |  $1470$  |
| EUR 3 | $100\times 120\times 14.4$ |  $1920$  |
| EUR 6 |  $80\times 60\times 14.4$  |   $500$  |

Container ISO:

|   |  $L\times W\times H (cm)$  | Load (kg) |
|:-:|:--------------------------:|:--------:|
| 1A | $233\times 1200\times 220$ |  $26480$ |
| 1C |  $233\times 587\times 220$ |  $28180$ |

Notes:
- Mean number of products per order: 75-100
- Different pallet types in the same order
- Same container type in the same order
- Item rotation not allowed

In [17]:
model = cp_model.CpModel()
max_pallets = ordered_products
Corner = namedtuple("Corner", "x y z")

# Variables
# product_in_pallet[i, j] = 1 if product i is packed in pallet j
product_in_pallet = {}
for i in range(ordered_products):
    for j in range(max_pallets):
        product_in_pallet[(i, j)] = model.NewBoolVar('product_in_pallet_%i_%i' % (i, j))
        
# pallet[i] = 1 if pallet i is used
pallet = {}
for i in range(max_pallets):
    pallet[i] = model.NewBoolVar('pallet_%i' % i)

# blb_corners[i] = [(x_s, x_e), (y_s, y_e), (z_s, z_e)] if product i has blb corner (x_s, y_s, z_s)
blb_corners_start = {}
blb_corners_end = {}
blb_corners_int = {}
for i in range(ordered_products):
    x_s = model.NewIntVar(0, pallet_lenght - int(order.iloc[i].lenght), 'blb_corner_x_start_%i' % i)
    y_s = model.NewIntVar(0, pallet_width - int(order.iloc[i].width), 'blb_corner_y_start_%i' % i)
    z_s = model.NewIntVar(0, max_product_height - int(order.iloc[i].height), 'blb_corner_z_start_%i' % i)
    blb_corners_start[i] = Corner(x_s, y_s, z_s)
    
    x_e = model.NewIntVar(int(order.iloc[i].lenght), pallet_lenght, 'blb_corner_x_end_%i' % i)
    y_e = model.NewIntVar(int(order.iloc[i].width), pallet_width, 'blb_corner_y_end_%i' % i)
    z_e = model.NewIntVar(int(order.iloc[i].height), max_product_height, 'blb_corner_z_end_%i' % i)
    blb_corners_end[i] = Corner(x_e, y_e, z_e)
    
    blb_corners_int[i] = [
        model.NewIntervalVar(x_s, int(order.iloc[i].lenght), x_e, 'blb_corner_x_int_%i' % i),
        model.NewIntervalVar(y_s, int(order.iloc[i].width), y_e, 'blb_corner_y_int_%i' % i),
        model.NewIntervalVar(z_s, int(order.iloc[i].height), z_e, 'blb_corner_z_int_%i' % i)
    ]
    
# prec[i, j, d] = 1 if product i precedes product j on dimension d 
prec = {}
for i in range(ordered_products):
    for j in range(ordered_products):
        if i != j:
            prec[i, j] = [
                model.NewBoolVar('%i_before_%i_x' % (i, j)),
                model.NewBoolVar('%i_before_%i_y' % (i, j)),
                model.NewBoolVar('%i_before_%i_z' % (i, j))
            ]
        
    
# Constraints
# Each product must be in exactly one pallet
for i in range(ordered_products):
    model.Add(sum(product_in_pallet[i, p] for p in range(max_pallets)) == 1)
    
# Precedence constraints
# prec[i, j, d] == 1 if product i precedes product j on dimension d 
#                    and both products are in the same pallet
for p in range(max_pallets):
    for i in range(ordered_products):
        for j in range(ordered_products):
            if i != j:
                model.Add(blb_corners_start[i].x + int(order.iloc[i].lenght) <= blb_corners_start[j].x).OnlyEnforceIf(
                    [prec[i, j][0], product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].x + int(order.iloc[i].lenght) > blb_corners_start[j].x).OnlyEnforceIf(
                    [prec[i, j][0].Not(), product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].y + int(order.iloc[i].width) <= blb_corners_start[j].y).OnlyEnforceIf(
                    [prec[i, j][1], product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].y + int(order.iloc[i].width) > blb_corners_start[j].y).OnlyEnforceIf(
                    [prec[i, j][1].Not(), product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].z + int(order.iloc[i].height) <= blb_corners_start[j].z).OnlyEnforceIf(
                    [prec[i, j][2], product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].z + int(order.iloc[i].height) > blb_corners_start[j].z).OnlyEnforceIf(
                    [prec[i, j][2].Not(), product_in_pallet[i, p], product_in_pallet[j, p]]
                )

# Constraint to ensure that there is at most one spatial relationship 
# between products i and j along each dimension, if they are on the same pallet
for p in range(max_pallets):
    for i in range(ordered_products):
        for j in range(ordered_products):
            if i != j:
                model.Add(prec[i, j][0] + prec[j, i][0] <= 1).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(prec[i, j][1] + prec[j, i][1] <= 1).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(prec[i, j][2] + prec[j, i][2] <= 1).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(prec[i, j][0] + prec[j, i][0] + prec[i, j][1] + prec[j, i][1] + prec[i, j][2] + prec[j, i][2] >= 1).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )
    
# Non-overlapping constraint
for p in range(max_pallets):
    for i in range(ordered_products):
        for j in range(ordered_products):
            if i != j:
                model.Add(blb_corners_start[i].x + int(order.iloc[i].lenght) <= blb_corners_start[j].x + pallet_lenght * (1 - prec[i, j][0])).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].y + int(order.iloc[i].width) <= blb_corners_start[j].y + pallet_width * (1 - prec[i, j][1])).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )
                model.Add(blb_corners_start[i].z + int(order.iloc[i].height) <= blb_corners_start[j].z + max_product_height * (1 - prec[i, j][2])).OnlyEnforceIf(
                    [product_in_pallet[i, p], product_in_pallet[j, p]]
                )

# The amount packed in each pallet cannot exceed its liquid volume
for p in range(max_pallets):
    model.Add(
        sum(
            product_in_pallet[i, p] * 
            int(order.iloc[i].lenght) * 
            int(order.iloc[i].width) * 
            int(order.iloc[i].height) 
            for i in range(ordered_products)
        ) <= pallet[p] * pallet_lenght * pallet_width * max_product_height
    )

# The amount packed in each pallet cannot exceed its maximum load
for p in range(max_pallets):
    model.Add(
        sum(
            product_in_pallet[i, p] * 
            int(order.iloc[i].weight) 
            for i in range(ordered_products)
        ) <= pallet[p] * pallet_load
    )

# If at least one product is in a pallet, then that pallet is used
for p in range(max_pallets):
    for i in range(ordered_products):
        model.AddImplication(product_in_pallet[i, p], pallet[p])

"""
for p in range(max_pallets):
    for i in range(ordered_products):
        x_zero = model.NewBoolVar(f"{i}_{p}_cxzero")
        model.Add(blb_corners_start[i].x == 0).OnlyEnforceIf(x_zero)
        y_zero = model.NewBoolVar(f"{i}_{p}_cyzero")
        model.Add(blb_corners_start[i].y == 0).OnlyEnforceIf(y_zero)
        z_zero = model.NewBoolVar(f"{i}_{p}_czzero")
        model.Add(blb_corners_start[i].z == 0).OnlyEnforceIf(z_zero)
        xyz_zero = model.NewBoolVar(f"{i}_{p}_c_xyz_zero")
        model.Add(xyz_zero == 1).OnlyEnforceIf([x_zero, y_zero, z_zero])
        
        corners = [xyz_zero]
        for j in range(ordered_products):
            if i != j:
                x_corner = model.NewBoolVar(f"{i}_{j}_{p}_cxc")
                model.Add(
                    blb_corners_start[i].x == blb_corners_start[j].x + int(order.iloc[j].lenght)
                ).OnlyEnforceIf([x_corner, product_in_pallet[i, p], product_in_pallet[j, p]])
                model.Add(
                    blb_corners_start[i].x != blb_corners_start[j].x + int(order.iloc[j].lenght)
                ).OnlyEnforceIf([x_corner.Not(), product_in_pallet[i, p], product_in_pallet[j, p]])
                
                y_corner = model.NewBoolVar(f"{i}_{j}_{p}_cyc")
                model.Add(
                    blb_corners_start[i].y == blb_corners_start[j].y
                ).OnlyEnforceIf([y_corner, product_in_pallet[i, p], product_in_pallet[j, p]])
                model.Add(
                    blb_corners_start[i].y != blb_corners_start[j].y
                ).OnlyEnforceIf([y_corner.Not(), product_in_pallet[i, p], product_in_pallet[j, p]])
                
                z_corner = model.NewBoolVar(f"{i}_{j}_{p}_czc")
                model.Add(
                    blb_corners_start[i].z == blb_corners_start[j].z
                ).OnlyEnforceIf([z_corner, product_in_pallet[i, p], product_in_pallet[j, p]])
                model.Add(
                    blb_corners_start[i].z != blb_corners_start[j].z
                ).OnlyEnforceIf([z_corner.Not(), product_in_pallet[i, p], product_in_pallet[j, p]])
                
                xyz_corner = model.NewBoolVar(f"{i}_{j}_{p}_cxyzc")
                model.Add(xyz_corner == 1).OnlyEnforceIf([x_corner, y_corner, z_corner])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner.Not(), y_corner, z_corner])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner.Not(), y_corner, z_corner.Not()])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner.Not(), y_corner.Not(), z_corner])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner.Not(), y_corner.Not(), z_corner.Not()])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner, y_corner.Not(), z_corner])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner, y_corner.Not(), z_corner.Not()])
                model.Add(xyz_corner == 0).OnlyEnforceIf([x_corner, y_corner, z_corner.Not()])
                
                corners += [xyz_corner]
        
        model.AddBoolOr(corners)
"""
            
# Search by decreasing volume
volumes = [int(order.iloc[i].lenght) * int(order.iloc[i].width) * int(order.iloc[i].height) for i in range(ordered_products)]
sorted_indexes = np.flip(np.argsort(volumes))
model.AddDecisionStrategy(
    [blb_corners_start[i].z for i in sorted_indexes],
    cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE
)

# Minimize the number of used pallets
model.Minimize(
    sum(pallet[p] for p in range(max_pallets))
    #sum(pallet[p] * max(blb_corners_start[i].z for i in range(ordered_products)) for p in range(max_pallets))
    #max([blb_corners_start[i].z + int(order.iloc[i].height) for i in range(ordered_products)])
)

TODO:
- Search by weight
- Check non-overlap constraint (min values for blb corners)
- Products don't fly constraint
    - Check about corner/extreme points (how to access list of already assigned variables in or-tools?)
- Time limit
- Superitems/layers

In [273]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 120.0
#solver.parameters.search_branching = cp_model.FIXED_SEARCH
start_solve_time = time()
status = solver.Solve(model)
solve_time = time() - start_solve_time
print(f"Number of variables: {model.NumVariables()}")
print(f'Solve status: {solver.StatusName(status)}')
print(f'Solve time: {solve_time}')

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print(f'Number of pallets: {solver.ObjectiveValue()}')
    print(f'Pallet dimensions: {pallet_lenght}x{pallet_width}x{max_product_height} mm')
    print(f'Pallet maximum load: {pallet_load} kg')
    print(f'Pallet liquid volume: {get_liquid_volume([[pallet_lenght, pallet_width, max_product_height]])} m3')
    print()
    for j in range(max_pallets):
        if solver.Value(pallet[j]):
            pallet_items, pallet_blbs, pallet_weights, pallet_dims = [], [], [], []
            print(f'Pallet #{j}')
            for i in range(ordered_products):
                if solver.Value(product_in_pallet[i, j]) > 0:
                    pallet_items.append(i)
                    pallet_blbs.append((
                        solver.Value(blb_corners_start[i].x),
                        solver.Value(blb_corners_start[i].y),
                        solver.Value(blb_corners_start[i].z)
                    ))
                    pallet_weights.append(order.iloc[i].weight)
                    pallet_dims.append([order.iloc[i].lenght, order.iloc[i].width, order.iloc[i].height])
                    print(f"\tProduct {i}")
                    print(f"\t\tBottom-left-back corner: {pallet_blbs[-1]}")
                    print(f"\t\tDimensions: {pallet_dims[-1][0]}x{pallet_dims[-1][1]}x{pallet_dims[-1][2]} mm")
                    print(f"\t\tWeight: {pallet_weights[-1]} kg")
            
            print(f'Total pallet weight: {sum(pallet_weights)} kg')
            print(f'Total pallet liquid volume: {get_liquid_volume(pallet_dims)} m3') 
            plot_pallet(
                order, 
                pallet_items,
                pallet_blbs, 
                pallet_lenght, 
                pallet_width, 
                max_product_height
            )
            print()
    print(solver.ResponseStats())
elif status == cp_model.MODEL_INVALID:
    print(model.Validate())

AttributeError: 'CpModel' object has no attribute 'NumVariables'

In [147]:
[
    (i, j, solver.Value(prec[i, j][0]), solver.Value(prec[i, j][1]), solver.Value(prec[i, j][2]))
    for i in range(ordered_products) for j in range(ordered_products) if i!=j
]

[(0, 1, 1, 1, 0),
 (0, 2, 1, 1, 0),
 (0, 3, 0, 1, 1),
 (0, 4, 0, 1, 1),
 (0, 5, 0, 1, 1),
 (0, 6, 0, 1, 1),
 (0, 7, 0, 1, 0),
 (0, 8, 1, 0, 0),
 (0, 9, 0, 1, 0),
 (0, 10, 1, 0, 0),
 (0, 11, 1, 0, 0),
 (0, 12, 1, 1, 1),
 (0, 13, 0, 1, 0),
 (0, 14, 1, 1, 0),
 (0, 15, 0, 0, 0),
 (0, 16, 1, 1, 0),
 (0, 17, 1, 1, 0),
 (0, 18, 0, 1, 0),
 (0, 19, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (1, 2, 0, 0, 0),
 (1, 3, 0, 0, 1),
 (1, 4, 0, 0, 1),
 (1, 5, 0, 0, 1),
 (1, 6, 0, 0, 1),
 (1, 7, 0, 1, 0),
 (1, 8, 0, 0, 0),
 (1, 9, 0, 0, 0),
 (1, 10, 0, 0, 0),
 (1, 11, 0, 0, 0),
 (1, 12, 0, 0, 1),
 (1, 13, 0, 0, 0),
 (1, 14, 0, 0, 0),
 (1, 15, 0, 0, 0),
 (1, 16, 0, 0, 0),
 (1, 17, 0, 0, 0),
 (1, 18, 0, 0, 0),
 (1, 19, 0, 0, 0),
 (2, 0, 0, 0, 1),
 (2, 1, 0, 0, 1),
 (2, 3, 0, 0, 1),
 (2, 4, 0, 0, 1),
 (2, 5, 0, 0, 1),
 (2, 6, 0, 0, 1),
 (2, 7, 0, 1, 1),
 (2, 8, 0, 0, 1),
 (2, 9, 0, 0, 1),
 (2, 10, 0, 0, 1),
 (2, 11, 0, 0, 1),
 (2, 12, 0, 0, 1),
 (2, 13, 0, 0, 0),
 (2, 14, 0, 0, 0),
 (2, 15, 0, 0, 1),
 (2, 16, 0, 0, 1),
 

In [366]:
model._CpModel__model

variables {
  name: "product_in_pallet_0_0"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_1"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_2"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_3"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_4"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_5"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_6"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_7"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_8"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_0_9"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_1_0"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_1_1"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_1_2"
  domain: 0
  domain: 1
}
variables {
  name: "product_in_pallet_1_3"
  domain: 0
  domain: 1
}
variables {
  name: 